In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split, ConcatDataset, Subset

In [2]:
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets_scaled.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [3]:
batch_size = 128 
loaded_train_loader = DataLoader(loaded_train_dataset, batch_size = batch_size) 
loaded_val_loader = DataLoader(loaded_val_dataset, batch_size = batch_size) 
loaded_test_loader = DataLoader(loaded_test_dataset, batch_size = batch_size, shuffle=False) 

In [4]:
def metrics_output(preds,labels):
    true_labels = np.array(labels)
    predicted_probs = np.array(preds)
    binary_predictions = (predicted_probs >= 0.5).astype(int)
    auc = roc_auc_score(true_labels, predicted_probs)
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = accuracy_score(true_labels, binary_predictions)
    f1 = f1_score(true_labels, binary_predictions)
    mcc = matthews_corrcoef(true_labels, binary_predictions)  
    return (auc, sensitivity, specificity, accuracy, f1, mcc)

In [5]:
class Net_conv(nn.Module):
    def __init__(self, input_length):
        super(Net_conv, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.1)  
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout(0.1) 
        flattened_size = 64 * input_length
        self.fc1 = nn.Linear(flattened_size, 64)
        self.dropout3 = nn.Dropout(0.1)  
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.dropout1(x)  
        x = self.relu(self.conv2(x))
        x = self.dropout2(x)  
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) 
        x = self.dropout3(x)  
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [14]:
model = Net_conv(input_length=24) # revise
criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 10

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_dataset = ConcatDataset([loaded_train_dataset, loaded_val_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

for fold_idx, (train_index, val_index) in enumerate(kf.split(train_loader.dataset)):
    predicted_probabilities = []
    true_labels = []
    train_loader_fold = DataLoader(Subset(train_loader.dataset, train_index), batch_size=batch_size)
    val_loader_fold = DataLoader(Subset(train_loader.dataset, val_index), batch_size=batch_size)
    model = Net_conv(input_length=24) # revise
    
    for epoch in range(num_epochs):
        model = model.train()
        for batch_idx, (features, targets) in enumerate(train_loader_fold):     
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets) 
            loss.backward()
            optimizer.step()

    with torch.set_grad_enabled(False):
        for batch_idx, (features, targets) in enumerate(val_loader_fold):
            logits = model(features.float())
            probabilities = logits
            predicted_probabilities.extend(probabilities.tolist())
            true_labels.extend(targets.tolist())
    #每一个都储存（没有必要，只要最后一次的就好）
    #torch.save(model.state_dict(), f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/CNN/CNN_{fold_idx}.pth')
    #np.save(f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/CNN/cnn_y_val_pred_{fold_idx}.npy', predicted_probabilities)
    #np.save(f'/Users/jiaming/Desktop/Lab2/datas/ROC/models/CNN/cnn_y_val_{fold_idx}.npy', true_labels)

In [16]:
# 只储存了最后一次
torch.save(model.state_dict(), '/Users/jiaming/Desktop/Lab2/datas/ROC/models/CNN/CNN.pth')

In [17]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities, true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.5053854875283447 0.9591836734693877 0.027777777777777776 0.470873786407767 0.632996632996633 -0.03594004467332665


In [18]:
# 只储存了最后一次
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/val/CNN/cnn_y_val_pred.npy', predicted_probabilities)
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/val/CNN/cnn_y_val.npy', true_labels)

In [19]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_idx, (features, targets) in enumerate(loaded_test_loader):
        logits = model(features.float())
        probabilities = logits
        predicted_probabilities.extend(probabilities.tolist()) 
        true_labels.extend(targets.tolist())

In [20]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.5874218374218375 0.9621212121212122 0.015873015873015872 0.5 0.6631853785900783 -0.06770534672403188


In [21]:
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/test/CNN/cnn_y_test_pred.npy', predicted_probabilities)
np.save('/Users/jiaming/Desktop/Lab2/datas/ROC/test/CNN/cnn_y_test.npy', true_labels)